# Verification of mortality effects

- Parameter variation `VerificationEFfects` in Anylogic
- 90 replicates per iteration or scenario

In [444]:
library(data.table)
library(xtable)
library(survival)
library(texreg)
library(fmsb)
source("../src/utils.R")

In [463]:
# life table verification LE using mortality rates reported by CDC
mx = c(567.0,24.3 ,11.6 ,15.5 ,51.5 ,51.5 ,95.6 ,121.0 ,145.4 ,173.8 ,218.4,
    313.2 ,488.0 ,736.5 ,1050.2 ,1473.5 ,2206.9 ,3517.8 ,5871.7,13573.6)
mx = mx / 100000
le = lifetable(mx, ns = c(1, 4, rep(5, 2), 3, 2, rep(5, 14)))
le[1, "ex"]

[1] 79.02566

In [446]:
# read data files
m = fread("../output/data/mortality-verification-effects.csv")
p = fread("../output/data/parameters-verification-effects.csv")
e = fread("../output/data/environment-verification-effects.csv")

# select parameters of interest
parameters = c("mortality_fake_exp_coeff")
setorderv(p, parameters)

# redefine iteration and replicate indexes
p[, niteration := .GRP, by = parameters]
p[, nreplicate := 1:.N, by = niteration]
np = p[, c("iteration", "replicate", "niteration", "nreplicate", parameters), with = FALSE]
m = merge(m, np, by = c("iteration", "replicate"))
summary(np[niteration == 3, nreplicate])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00   23.25   45.50   45.50   67.75   90.00 

In [447]:
anyDuplicated(m[niteration == 1 & nreplicate == 1, id])
table(m[niteration == 1 & nreplicate == 1, generation])

[1] 0


  20   25   30 
3110 3027 3055 

In [448]:
summary(e$nsi)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2553  0.3387  0.3590  0.3579  0.3787  0.4621 

In [449]:
mean(m[niteration == 1, age_death])

[1] 76.59931

In [450]:
head(m[age_death == 0])

iteration,replicate,model_time,id,fertility_control,parent_id,generation,birthdate,age_death,income_group,...,county_rank_correlation,county_rank_slope,total_rank_slope_exposure,rank_slope_exposure18,total_rank_cor_exposure,total_z_income_exposure,z_income_exposure30,niteration,nreplicate,mortality_fake_exp_coeff
<int>,<int>,<dbl>,<int>,<lgl>,<dbl>,<int>,<dbl>,<dbl>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,1,556.1525,71612,FALSE,67592,20,555.3090,0,1,...,0.1981626,0.2907863,0.2907863,0,0.1981626,1.4906075,0,1,1,0
1,1,565.4195,72749,FALSE,70910,20,564.9589,0,1,...,0.3197768,0.3602353,0.3602353,0,0.3197768,-1.3365796,0,1,1,0
1,1,573.5341,73709,FALSE,69791,20,573.5004,0,5,...,0.1069098,0.1565108,0.1565108,0,0.1069098,-0.4839215,0,1,1,0
1,1,583.4157,74831,FALSE,71610,20,582.7497,0,1,...,0.2178619,0.1965774,0.1965774,0,0.2178619,-0.3419976,0,1,1,0
1,1,584.0900,74950,FALSE,73152,20,583.6599,0,1,...,0.2698617,0.2500411,0.2500411,0,0.2698617,-0.4283472,0,1,1,0
1,1,586.9844,75321,FALSE,71017,20,586.6672,0,3,...,0.1478664,0.2195749,0.2195749,0,0.1478664,-0.6843235,0,1,1,0


In [451]:
table(m$fertility_control)


  FALSE 
2520920 

# Exposure relative income mobility -> Mortality risk = 0.0

To explore correlation of the data produced by the ABM: 

- `total_rank_slope_exposure` and `rank_slope_exposure18`: exposure measures of place's rank-rank slopes (relative mobility)
- `income_group_mobility`: individual mobilityl, absolute value of (`income_group` - `parent_income_group`)
- `total_z_income_exposure`: exposure of standardized income

In [452]:
t = m[niteration == 1]
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, status := 1] # there is no censoring
t[, individual_income_mobility := abs(income_group_mobility)]
mean(t$age_death)

[1] 76.59931

In [453]:
# just an indicator of change
table(t$individual_income_mobility)


     0      1      2      3      4 
232452 302884 176116  86818  31423 

In [454]:
# average number of county changes (residential moves)
mean(t$nmoves)

[1] 2.351763

In [455]:
cor(t[, .(total_rank_slope_exposure, rank_slope_exposure18, county_rank_slope, individual_income_mobility, income, 
    county_mean_income, smoker, age_death, total_z_income_exposure)])

,total_rank_slope_exposure,rank_slope_exposure18,county_rank_slope,individual_income_mobility,income,county_mean_income,smoker,age_death,total_z_income_exposure
total_rank_slope_exposure,1.00000000,0.55139191,0.620336542,-0.04670284,-0.02514040,-0.04993939,0.01697095,-0.011084899,-0.05044790
rank_slope_exposure18,0.55139191,1.00000000,0.109379817,0.01191604,-0.01992163,-0.02932892,0.02947758,0.137941495,-0.03271842
county_rank_slope,0.62033654,0.10937982,1.000000000,-0.02440658,-0.01537423,-0.03972620,0.01098335,-0.006668863,-0.03415695
individual_income_mobility,-0.04670284,0.01191604,-0.024406582,1.00000000,0.14191149,0.11026488,-0.03570979,0.072102745,0.08341970
income,-0.02514040,-0.01992163,-0.015374227,0.14191149,1.00000000,0.47059282,-0.15164755,0.099264313,0.37977934
county_mean_income,-0.04993939,-0.02932892,-0.039726199,0.11026488,0.47059282,1.00000000,-0.12939268,0.103481953,0.78237204
smoker,0.01697095,0.02947758,0.010983353,-0.03570979,-0.15164755,-0.12939268,1.00000000,-0.216448196,-0.13310991
age_death,-0.01108490,0.13794149,-0.006668863,0.07210275,0.09926431,0.10348195,-0.21644820,1.000000000,0.12530725
total_z_income_exposure,-0.05044790,-0.03271842,-0.034156947,0.08341970,0.37977934,0.78237204,-0.13310991,0.125307255,1.00000000


In [456]:
# survival model 
m1 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure, data = t)
m2 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + individual_income_mobility, data = t)
m3 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker, data = t)
m4 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income, data = t)
m5 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4, m5)))


                            Model 1          Model 2          Model 3          Model 4          Model 5        
---------------------------------------------------------------------------------------------------------------
total_rank_slope_exposure          0.21 ***         0.18 ***         0.10 ***         0.04 *           0.04 *  
                                  (0.02)           (0.02)           (0.02)           (0.02)           (0.02)   
individual_income_mobility                         -0.03 ***                                                   
                                                   (0.00)                                                      
lincome                                                             -0.04 ***        -0.02 ***        -0.02 ***
                                                                    (0.00)           (0.00)           (0.00)   
smokerTRUE                                                           1.00 ***         0.99 ***         

# Exposure relative income mobility -> Mortality risk = 0.3

In [457]:
t = m[niteration == 2]
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, status := 1] # there is no censoring
t[, individual_income_mobility := abs(income_group_mobility)]
mean(t$age_death)

[1] 75.5347

In [458]:
cor(t[, .(total_rank_slope_exposure, rank_slope_exposure18, county_rank_slope, individual_income_mobility, income, 
    county_mean_income, smoker, age_death, total_z_income_exposure)])

,total_rank_slope_exposure,rank_slope_exposure18,county_rank_slope,individual_income_mobility,income,county_mean_income,smoker,age_death,total_z_income_exposure
total_rank_slope_exposure,1.00000000,0.55673223,0.623743470,-0.04606203,-0.02868710,-0.05715794,0.013829416,-0.02496618,-0.05639704
rank_slope_exposure18,0.55673223,1.00000000,0.114364024,0.01334984,-0.02313781,-0.03472378,0.030899671,0.13601612,-0.03811696
county_rank_slope,0.62374347,0.11436402,1.000000000,-0.02337915,-0.01876888,-0.04588804,0.004737817,-0.01104675,-0.03732342
individual_income_mobility,-0.04606203,0.01334984,-0.023379147,1.00000000,0.13834372,0.10820115,-0.034199867,0.07636763,0.07985654
income,-0.02868710,-0.02313781,-0.018768883,0.13834372,1.00000000,0.46645140,-0.152995830,0.09711124,0.37377100
county_mean_income,-0.05715794,-0.03472378,-0.045888040,0.10820115,0.46645140,1.00000000,-0.129658464,0.10478049,0.77804432
smoker,0.01382942,0.03089967,0.004737817,-0.03419987,-0.15299583,-0.12965846,1.000000000,-0.21269315,-0.13220483
age_death,-0.02496618,0.13601612,-0.011046746,0.07636763,0.09711124,0.10478049,-0.212693152,1.00000000,0.12426664
total_z_income_exposure,-0.05639704,-0.03811696,-0.037323422,0.07985654,0.37377100,0.77804432,-0.132204834,0.12426664,1.00000000


In [459]:
# survival model 
m1 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure, data = t)
m2 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + individual_income_mobility, data = t)
m3 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker, data = t)
m4 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income, data = t)
m5 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4, m5)))


                            Model 1          Model 2          Model 3          Model 4          Model 5        
---------------------------------------------------------------------------------------------------------------
total_rank_slope_exposure          0.46 ***         0.44 ***         0.40 ***         0.33 ***         0.32 ***
                                  (0.02)           (0.02)           (0.02)           (0.02)           (0.02)   
individual_income_mobility                         -0.03 ***                                                   
                                                   (0.00)                                                      
lincome                                                             -0.04 ***        -0.02 ***        -0.02 ***
                                                                    (0.00)           (0.00)           (0.00)   
smokerTRUE                                                           1.00 ***         0.98 ***         

# Exposure relative income mobility -> Mortality risk = 0.5

In [460]:
t = m[niteration == 3]
t[, lincome := logIncome(income)]
t[, lcounty_income := logIncome(county_mean_income)]
t[, status := 1] # there is no censoring
t[, individual_income_mobility := abs(income_group_mobility)]
mean(t$age_death)

[1] 74.72194

In [461]:
cor(t[, .(total_rank_slope_exposure, rank_slope_exposure18, county_rank_slope, individual_income_mobility, income, 
    county_mean_income, smoker, age_death, total_z_income_exposure)])

,total_rank_slope_exposure,rank_slope_exposure18,county_rank_slope,individual_income_mobility,income,county_mean_income,smoker,age_death,total_z_income_exposure
total_rank_slope_exposure,1.00000000,0.55596962,0.62336887,-0.04982762,-0.02517057,-0.04927179,0.01282192,-0.03447694,-0.05243803
rank_slope_exposure18,0.55596962,1.00000000,0.11103125,0.01363719,-0.01837167,-0.02591718,0.02682990,0.13921100,-0.03056370
county_rank_slope,0.62336887,0.11103125,1.00000000,-0.02693797,-0.01529911,-0.03894025,0.00886383,-0.01947163,-0.03476024
individual_income_mobility,-0.04982762,0.01363719,-0.02693797,1.00000000,0.14652680,0.11826264,-0.03594378,0.07915445,0.08916638
income,-0.02517057,-0.01837167,-0.01529911,0.14652680,1.00000000,0.47567272,-0.15176550,0.09964285,0.38000338
county_mean_income,-0.04927179,-0.02591718,-0.03894025,0.11826264,0.47567272,1.00000000,-0.12988790,0.10612685,0.77604448
smoker,0.01282192,0.02682990,0.00886383,-0.03594378,-0.15176550,-0.12988790,1.00000000,-0.20787721,-0.13258969
age_death,-0.03447694,0.13921100,-0.01947163,0.07915445,0.09964285,0.10612685,-0.20787721,1.00000000,0.12588946
total_z_income_exposure,-0.05243803,-0.03056370,-0.03476024,0.08916638,0.38000338,0.77604448,-0.13258969,0.12588946,1.00000000


In [462]:
# survival model 
m1 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure, data = t)
m2 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + individual_income_mobility, data = t)
m3 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker, data = t)
m4 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income, data = t)
m5 = coxph(Surv(age_death, status) ~ total_rank_slope_exposure + lincome + smoker + lcounty_income + total_z_income_exposure, data = t)
cat(screenreg(list(m1, m2, m3, m4, m5)))


                            Model 1          Model 2          Model 3          Model 4          Model 5        
---------------------------------------------------------------------------------------------------------------
total_rank_slope_exposure          0.63 ***         0.59 ***         0.57 ***         0.51 ***         0.51 ***
                                  (0.02)           (0.02)           (0.02)           (0.02)           (0.02)   
individual_income_mobility                         -0.04 ***                                                   
                                                   (0.00)                                                      
lincome                                                             -0.04 ***        -0.02 ***        -0.02 ***
                                                                    (0.00)           (0.00)           (0.00)   
smokerTRUE                                                           0.99 ***         0.97 ***         